# Assignment 2 – Evaluating LLM Output Quality

# Introduction to Large Language Models and Prompt Engineering

**Course:** GenAI development and LLM applications


**Instructors:** Ori Shapira, Yuval Belfer

**Semester:** Summer
    
## Overview

This assignment provides a **hands‑on** experince with the world of LLM based systems evaluation: from understanding the business use-case and defining evaluation criterias in light of it. To performing human evaluation and dealing with the hardships of "non-objectivity", trough experimenting with **JLMs** (Judge Language Models).

Along the way you will explore the differnces between the two evaluation methods, thier advanteges and dis-advanteges and try to figure out how and when to use each further down your GenAI road.

## Learning Objectives

- **Define evaluation criteria** understand the importance of defining how to measure your system performance in a non closely defined problem.
- **Compare** manual vs. automatic common methods.
- **Drive improvement** through proper evaluation, documentation and change cycles.
- **Design** usable automatic evaluation pipeline.

## Prerequisites
- Basic Python knowledge
- Familiarity with Jupyter notebooks
- Internet connection for API calls

# Part 1 - Human evaluation

## 1  Setup

In [1]:
!pip -q install --upgrade "transformers[torch]" datasets accelerate bitsandbytes --progress-bar off

## 2  Business use case – Generate Product Descriptions
Many e‑commerce sites need engaging **product descriptions**. Given structured attributes (name, category, features, color, price), your model should craft a persuasive, 50‑90‑word description.

In [3]:
# Load the product dataset
import pandas as pd

dataset_path = "Assignment_02_product_dataset.csv"  # ensure the file is uploaded
df_products = pd.read_csv(dataset_path)
print(f"Loaded {len(df_products)} products")
df_products.head()

Loaded 50 products


,product_name,Product_attribute_list,material,warranty
0,Apple iPhone 15 Pro,"features: A17 Pro chip, 120 Hz ProMotion displ...","titanium frame, Ceramic Shield glass",1‑year limited warranty
1,Samsung Galaxy S24 Ultra,"features: 200 MP camera, S‑Pen support, 120 Hz...","Armor Aluminum frame, Gorilla Glass Victus",1‑year limited warranty
2,Google Pixel 8 Pro,"features: Tensor G3 chip, Magic Eraser, 50 MP ...","matte glass back, aluminum frame",1‑year limited warranty
3,Sony WH‑1000XM5 Headphones,"features: active noise cancelling, 30 hr batte...",synthetic leather earcups,1‑year limited warranty
4,Bose QuietComfort Ultra Earbuds,"features: CustomTune sound calibration, ANC, I...",silicone ear tips,1‑year limited warranty


## 3  Evaluation criteria
| Criterion | Description | Rating |
|-----------|-------------|--------|
| **Fluency** | Natural, easy‑to‑read sentences | good / ok / bad |
| **Grammar** | Correct spelling & punctuation | good / ok / bad |
| **Tone** | Matches friendly, credible sales voice | good / ok / bad |
| **Length** | 50‑90 words | good / ok / bad |
| **Grounding** | Sticks to provided attributes only | good / ok / bad |
| **Latency** | Time to first byte / full response | good / ok / bad  (based on avg. time per call)|
| **Cost** | Relative inference or API cost per 1K tokens | good / ok / bad (based on avg. price per cal)|

**Define your rubric:**
1. For each criterion, spell out what qualifies as **good**, **ok**, and **bad** to minimize subjectivity (e.g. for *Length*: good = 50‑90 words, ok = 40‑49 or 91‑110 words, bad = outside that range).
2. Decide the **cumulative pass bar**—for instance, at least three *good* ratings and no *bad* ratings overall.
3. Establish **go / no‑go rules**—e.g. if *Grounding* is *bad* the description is automatically rejected, regardless of other scores.

## 4  Prompt

💡 **Prompt‑engineering tip:**
Feel free to iterate on the prompt to maximize output quality. You can:
- Add a **system message** that defines writing style, brand voice, or formatting rules.
- Provide **one or two high‑quality examples** (few‑shot) of attribute→description pairs.
- Include explicit constraints (word count, tone adjectives, banned phrases).
- Experiment with phrases like *"Think step‑by‑step"* or *"First reason, then answer"*.

Document any changes you make and observe how they influence the evaluation scores.

In [4]:
prompt_tmpl = (
    "You are a copywriter for an online store. Using the product attributes, "
    "write an engaging product description (50–90 words).\n\n"
    "Product name: {product_name}\nFeatures: {Product_attribute_list}\Material: {material}\nWarranty: {warranty}\n\n"
    "Description:"
)

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
/var/folders/8k/xt0vhllx4jj6srjchtkrplr40000gn/T/ipykernel_48965/3038005655.py:4: SyntaxWarning: invalid escape sequence '\M'
  "Product name: {product_name}\nFeatures: {Product_attribute_list}\Material: {material}\nWarranty: {warranty}\n\n"


## 5  Run a medium‑size model (≤ 30 B parameters)

Choose **one or more** of the options below:

**A. Hugging Face checkpoint** (local inference) – already configured in the code cell that follows.

**B. OpenAI model** – call an OpenAI hosted model (e.g. `gpt‑4o`, `gpt‑4‑turbo`, `gpt‑3.5‑turbo`). Implement `call_openai(prompt: str) -> str` in a separate utility cell and then run the snippet.

**C. Google Gemini model** – call a Gemini endpoint (e.g. `gemini‑1.5‑pro`). Implement `call_gemini(prompt: str) -> str` similarly.

> ⚠️ Make sure you have your API keys set as environment variables or passed securely.


**Latency & cost tracking**
- Your `call_*` functions should return a **dict** with keys:
  - `text` – generated description (string)
  - `latency_ms` – end‑to‑end generation time in milliseconds
  - `input_tokens` – tokens sent to the model (**IF YOU ADDED A SYS PROMPT ADD IT TO THE CALCULATION**)
  - `output_tokens` – tokens received from the model
- Below, a helper `call_hf()` shows how to compute these metrics for a Hugging Face model. You must add equivalent tracking inside `call_openai()` and `call_gemini()`.


**FOR COLAB USERS**

You can set your HF_TOKEN secret in Colab, please follow these steps:

1. Click on the "🔑" icon in the left-hand sidebar in Colab. This opens the Secrets manager.
2. Click on "New secret".
3. In the "Name" field, type HF_TOKEN.
4. In the "Value" field, paste your Hugging Face access token (you can generate one from your Hugging Face account settings under "Access Tokens").
5. Make sure the "Notebook access" toggle is enabled for your notebook.
6. Close the Secrets manager.

In [5]:
# Set your Hugging Face access token
import os
os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"
print(os.environ.get("HF_TOKEN"))

YOUR_HF_TOKEN


In [6]:
import torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

assert torch.cuda.is_available(), "Switch Colab to GPU first"

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb = BitsAndBytesConfig(load_in_4bit=True)

# this will work with L4 GPU, if you have a different GPU, you may need to modify the code
hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    max_memory={0: "24GiB", "cpu": "30GiB"},
    quantization_config=bnb,
    low_cpu_mem_usage=True
)

hf_tok = AutoTokenizer.from_pretrained(model_id)

hf_gen = pipeline("text-generation",
               model=hf_model,
               tokenizer=hf_tok,
               max_new_tokens=120,
               do_sample=False)

/Users/kimchen/anaconda3/envs/new-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Switch Colab to GPU first

In [7]:
sample = dict(
    product_name="EverCool Water Bottle",
    Product_attribute_list="double-wall vacuum insulation; keeps drinks cold 24 h; leak-proof lid",
    material="stainless steel",
    warranty="lifetime warranty",
)

print(hf_gen(prompt_tmpl.format(**sample),
          return_full_text=False)[0]["generated_text"])

NameError: name 'hf_gen' is not defined

In [ ]:
# --- Option A: Hugging Face ---
# Ensure you implemented call_hf() in another cell (you can use the one from the previous assignment).

# Example with HF pipeline
def call_hf(prompt: str, model_id: str = "mistralai/Mistral-7B-Instruct-v0.3"):
    """Return dict with latency & token counts for HF pipeline."""
    # tokenizer = AutoTokenizer.from_pretrained(model_id)
    # model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)
    # generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=120)

    import time
    start = time.time()
    res= hf_gen(prompt,
          return_full_text=False)[0]["generated_text"]

    latency = (time.time() - start) * 1000  # ms
    # token counts via tokenizer
    input_tokens = len(hf_tok.encode(prompt))
    output_tokens = len(hf_tok.encode(res))
    return {
        "text": res,
        "latency_ms": latency,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
    }

response_hf = call_hf(prompt_tmpl.format(**sample))
print(response_hf)

In [ ]:
# --- Option B: OpenAI ---
# Ensure you implemented call_openai() in another cell (you can use the one from the previous assignment).
# from your_utils import call_openai  # example import if needed
# response_oai = call_openai(prompt, model_name="gpt-4o")
# print(response_oai)

In [8]:
import google.generativeai as genai
import time

In [9]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access variables
gemini_api_key = os.getenv("gemini_api_key")

In [ ]:
import openai
import time

def call_openai(prompt: str, model_name: str = "gpt-4o", api_key: str = None) -> dict:
    """
    Call OpenAI API and return generated text and usage metrics.

    Args:
        prompt (str): The user prompt.
        model_name (str): OpenAI model to use (default: "gpt-4o").
        api_key (str): OpenAI API Key (optional if already set globally).

    Returns:
        dict: {
            "text": generated description (str),
            "latency_ms": latency in milliseconds (float),
            "input_tokens": tokens sent (int),
            "output_tokens": tokens received (int)
        }
    """
    if api_key:
        openai.api_key = api_key

    start_time = time.time()

    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )

        end_time = time.time()
        latency_ms = (end_time - start_time) * 1000

        output = response.choices[0].message.content.strip()
        usage = response.usage

        return {
            "text": output,
            "latency_ms": latency_ms,
            "input_tokens": usage.prompt_tokens if usage else None,
            "output_tokens": usage.completion_tokens if usage else None
        }

    except Exception as e:
        print(f"OpenAI API Error: {e}")
        return {
            "text": "",
            "latency_ms": 0,
            "input_tokens": 0,
            "output_tokens": 0
        }


In [10]:
# --- Option C: Gemini ---
# --- Option C: Gemini ---
def call_gemini(prompt: str, model_name: str = "models/gemini-1.5-pro", api_key: str = gemini_api_key) -> dict:
    """
    Call Google Gemini API and return generated text and usage metrics.

    Args:
        prompt (str): The user prompt.
        model_name (str): Gemini model to use (default: "gemini-pro").
        api_key (str): Google API Key.

    Returns:
        dict: {
            "text": generated description (str),
            "latency_ms": latency in milliseconds (float),
            "input_tokens": tokens sent (int),
            "output_tokens": tokens received (int)
        }
    """

    if not api_key:
        raise ValueError("api_key must be provided for Google Gemini API")

    genai.configure(api_key=api_key)

    # Initialize model
    model = genai.GenerativeModel(model_name)

    # Measure start time
    start_time = time.time()

    try:
        response = model.generate_content(prompt)
        end_time = time.time()
        latency_ms = (end_time - start_time) * 1000

        text = response.text.strip() if hasattr(response, "text") else ""

        # Token accounting (Gemini SDK does not expose usage directly yet)
        input_tokens = len(prompt.split())
        output_tokens = len(text.split())

        return {
            "text": text,
            "latency_ms": latency_ms,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens
        }

    except Exception as e:
        print(f"Gemini API Error: {e}")
        return {
            "text": "",
            "latency_ms": 0,
            "input_tokens": 0,
            "output_tokens": 0
        }

In [15]:
# --- Batch generation helper (type‑safe) ---
from typing import Callable
import pandas as pd

from typing import Callable, Dict
import pandas as pd

def batch_generate(
    sample_df: pd.DataFrame,
    call_model_fn: Callable[[str], Dict[str, object]],
    prompt_template: str = prompt_tmpl,
) -> pd.DataFrame:
    """Generate descriptions and metrics for each row in *sample_df*.

    The model-calling function *must* return a dict with keys:
    - ``text`` (str) – generated description
    - ``latency_ms`` (float | None)
    - ``input_tokens`` (int | None)
    - ``output_tokens`` (int | None)
    """
    if not isinstance(sample_df, pd.DataFrame):
        raise TypeError("sample_df must be a pandas DataFrame")
    if not callable(call_model_fn):
        raise TypeError("call_model_fn must be callable")

    outputs = []
    for _, row in sample_df.iterrows():
        prompt = prompt_template.format(**row.to_dict())
        out = call_model_fn(prompt)
        if not isinstance(out, dict) or 'text' not in out:
            raise ValueError("call_model_fn must return a dict with at least a 'text' field")
        outputs.append(out)

    result_df = sample_df.copy()
    result_df["generated_description"] = [o["text"] for o in outputs]
    result_df["latency_ms"] = [o.get("latency_ms") for o in outputs]
    result_df["input_tokens"] = [o.get("input_tokens") for o in outputs]
    result_df["output_tokens"] = [o.get("output_tokens") for o in outputs]
    return result_df


demo_df = batch_generate(df_products[:5], call_gemini)
demo_df.head()

,product_name,Product_attribute_list,material,warranty,generated_description,latency_ms,input_tokens,output_tokens
0,Apple iPhone 15 Pro,"features: A17 Pro chip, 120 Hz ProMotion displ...","titanium frame, Ceramic Shield glass",1‑year limited warranty,Experience the blazing-fast speed of the Apple...,2322.148800,49,59
1,Samsung Galaxy S24 Ultra,"features: 200 MP camera, S‑Pen support, 120 Hz...","Armor Aluminum frame, Gorilla Glass Victus",1‑year limited warranty,Capture brilliance with the Samsung Galaxy S24...,2191.933155,48,57
2,Google Pixel 8 Pro,"features: Tensor G3 chip, Magic Eraser, 50 MP ...","matte glass back, aluminum frame",1‑year limited warranty,Experience the power of the Google Pixel 8 Pro...,2248.890877,47,60
3,Sony WH‑1000XM5 Headphones,"features: active noise cancelling, 30 hr batte...",synthetic leather earcups,1‑year limited warranty,Experience sound like never before with the So...,2106.420040,44,54
4,Bose QuietComfort Ultra Earbuds,"features: CustomTune sound calibration, ANC, I...",silicone ear tips,1‑year limited warranty,Experience world-class sound and silence with ...,1988.033056,42,54


## 6  Manual evaluation
Use `batch_generate()` to create a DataFrame of model outputs, then add blank rating columns for each criterion plus a `final_score` column. An Excel file is saved so you can fill scores offline or share with peers.

Steps:
1. Run the code cell below (adjust which `call_*` function you pass in).
2. Open the generated `assignment_03_evaluation_sheet.xlsx` and rate each row with **good / ok / bad**.
3. Add a rule for `final_score` (e.g., majority = good, fails if grounding = bad).


**Cost calculator**
Use the helper below to compute cost in USD based on token usage:
```python
outputs_df = add_cost_columns(outputs_df, input_price_per_m=1.5, output_price_per_m=2.0)
```
Set prices to **0** if you ran everything locally on Hugging Face.

In [18]:
# --- Cost computation helper ---
def add_cost_columns(df, input_price_per_m: float, output_price_per_m: float):
    """Add cost columns based on token counts.
    Args:
        df: DataFrame with `input_tokens` and `output_tokens`.
        input_price_per_m: $ per 1M input tokens.
        output_price_per_m: $ per 1M output tokens.
    Returns: DataFrame with extra `cost_usd` column.
    """
    if 'input_tokens' not in df or 'output_tokens' not in df:
        raise ValueError('Token columns missing; run batch_generate first')
    cost_input = df['input_tokens'] * (input_price_per_m / 1000000)
    cost_output = df['output_tokens'] * (output_price_per_m / 1000000)
    df = df.copy()
    df['cost_usd'] = (cost_input + cost_output).round(4)
    return df

# Example usage (set prices to 0 for HF local models):
outputs_df = add_cost_columns(demo_df, input_price_per_m=1.5, output_price_per_m=2.0)
outputs_df.head()

,product_name,Product_attribute_list,material,warranty,generated_description,latency_ms,input_tokens,output_tokens,cost_usd
0,Apple iPhone 15 Pro,"features: A17 Pro chip, 120 Hz ProMotion displ...","titanium frame, Ceramic Shield glass",1‑year limited warranty,Experience the blazing-fast speed of the Apple...,2322.148800,49,59,0.0002
1,Samsung Galaxy S24 Ultra,"features: 200 MP camera, S‑Pen support, 120 Hz...","Armor Aluminum frame, Gorilla Glass Victus",1‑year limited warranty,Capture brilliance with the Samsung Galaxy S24...,2191.933155,48,57,0.0002
2,Google Pixel 8 Pro,"features: Tensor G3 chip, Magic Eraser, 50 MP ...","matte glass back, aluminum frame",1‑year limited warranty,Experience the power of the Google Pixel 8 Pro...,2248.890877,47,60,0.0002
3,Sony WH‑1000XM5 Headphones,"features: active noise cancelling, 30 hr batte...",synthetic leather earcups,1‑year limited warranty,Experience sound like never before with the So...,2106.420040,44,54,0.0002
4,Bose QuietComfort Ultra Earbuds,"features: CustomTune sound calibration, ANC, I...",silicone ear tips,1‑year limited warranty,Experience world-class sound and silence with ...,1988.033056,42,54,0.0002


In [20]:
# --- Build evaluation sheet & export to Excel ---

#Update the prices according to the model you used, or leave them at 0 for HF local models
YOUR_MODEL_INPUT_PRICE_PER_M = 1.25
YOUR_MODEL_OUTPUT_PRICE_PER_M = 5
outputs_df = batch_generate(df_products, call_gemini)  # NOTE: change model function as needed

# Add rating columns (good/ok/bad)
rating_cols = ["fluency", "grammar", "tone", "length", "grounding", "latency", "cost", "final_score"]
for col in rating_cols:
    if col not in outputs_df:
        outputs_df[col] = ""

xlsx_path = "evaluated_assignment_03.xlsx"

# Add cost columns
outputs_df = add_cost_columns(outputs_df, YOUR_MODEL_INPUT_PRICE_PER_M, YOUR_MODEL_OUTPUT_PRICE_PER_M)

outputs_df.to_excel(xlsx_path, index=False)
print(f"Saved evaluation sheet → {xlsx_path} with {len(outputs_df)} rows")

Saved evaluation sheet → evaluated_assignment_03.xlsx with 50 rows


## 7  Improvement cycle

Now that you’ve established a baseline score in **Section 6**, iterate to achieve better results.

**Ideas to explore**
- **Prompt tuning** – rewrite the system/user prompts, add few‑shot examples, or enforce stricter constraints.
- **Model choice** – test a different checkpoint (larger ≠ always better), switch from HF to OpenAI or Gemini, or try a domain‑specific model.
- **Temperature / decoding params** – adjust `temperature`, `top_p`, `top_k`, or `max_new_tokens` to balance creativity vs. factuality.
- **Data preprocessing** – clean attribute text, expand abbreviations, or group similar products to feed additional context.
- **Post‑processing** – run grammar‑checking or length trimming after generation.
- **Ensembling / RAG** – combine outputs from two models or ground the prompt with retrieved copy from existing catalog listings.

Document each experiment in a brief bullet list:
1. **What you changed**
2. **Why you expected it to help**
3. **New evaluation scores**

💡 *Goal*: maximize the cumulative score according to your rubric while respecting the go/no‑go rules.

# Part 2 – Judging Language Models (JLMs)

In Part 1 we generated model outputs and evaluated the outputs according to a set of business criterion.
Now we will build **JLMs** that (try really hard to match what you did and) automatically *grade* those outputs.

## 1 Naïve JLM (same model as generation)

For the product‑description task there is **no single reference answer**.  
Your naïve JLM should simply *critique the description itself* using the same
LLM that produced it.

* Inputs: the generated product description (string) and any rubric bullets you
  think matter (e.g. factual accuracy, persuasiveness, tone).  
* Output: **any useful signal** – free‑text comments, a JSON blob, a (score, notes) tuple… your choice.

> **Deliverable:** implement `naive_jlm(description)` that calls the LLM once and
> returns its raw judgment output.

In [ ]:
judge_prompt_tmpl ="""
You are an expert evaluator of marketing copy. Based on the product information and the generated description below, evaluate the description using **only** these labels: "good", "ok", or "bad".

Rate each of the following categories:
- fluency
- grammar
- tone
- length (should be between 50–90 words)
- grounding (faithfulness to product attributes)
- latency (perceived delivery speed / verbosity)
- cost (efficiency of wording)
- final_score (overall quality)

**Product Details**
- Product name: {product_name}
- Key features: {Product_attribute_list}
- Material: {material}
- Warranty: {warranty}

**Generated Description**
{generated_description}

Return your evaluation in the following JSON format:

{{
  "fluency": "<good|ok|bad>",
  "grammar": "<good|ok|bad>",
  "tone": "<good|ok|bad>",
  "length": "<good|ok|bad>",
  "grounding": "<good|ok|bad>",
  "latency": "<good|ok|bad>",
  "cost": "<good|ok|bad>",
  "final_score": "<good|ok|bad>"
}}
"""


In [22]:
description = "Experience the pinnacle of innovation with the Apple iPhone 15 Pro.  Its powerful A17 Pro chip fuels a breathtaking 120Hz ProMotion display for incredibly smooth visuals.  The durable titanium frame and Ceramic Shield glass encase this compact powerhouse. Enjoy blazing-fast charging with USB-C. Comes with a 1-year limited warranty for peace of mind. Upgrade to pro."

In [23]:
# ------------------------- STUDENT TODO -------------------------
# Implement a simple self‑critique using the SAME model used for generation.
# No reference answer is available, so evaluate the description directly.

# IMPORTANT NOTE :use the same model as the one you used for generation
def naive_jlm(description: str):
    """Return a naive judgment (format of your choice) for a product description."""
    # Example pseudocode:
    prompt = """You are a product‑description critic. Evaluate the following description
     on clarity, appeal, and accuracy. Return JSON with 'strengths', 'weaknesses',
     and an overall 'rating' from 1‑5.\n\nDescription:\n{description}\nJSON:"""
    
    response = call_gemini(prompt.format(description=description))
    return response

naive_jlm(description)

{'text': '```json\n{\n  "strengths": [\n    "Highlights key features like the A17 Pro chip, ProMotion display, and durable construction.",\n    "Uses evocative language such as \'breathtaking,\' \'incredibly smooth,\' and \'blazing-fast.\'",\n    "Includes practical information like the USB-C charging and warranty.",\n    "Clear call to action: \'Upgrade to pro.\'"\n  ],\n  "weaknesses": [\n    "Lacks specific details about camera specs, storage options, and other important features.",\n    "Overuses superlatives like \'pinnacle\' and \'powerful\' without providing quantifiable support.",\n    "The phrase \'compact powerhouse\' could be contradictory and depends on the user\'s perspective.",\n    "Doesn\'t mention the price, which is a crucial factor for purchase decisions."\n  ],\n  "rating": 3\n}\n```',
 'latency_ms': 4060.133934020996,
 'input_tokens': 83,
 'output_tokens': 98}

## 2 Structured JLM – JSON per criterion

We want our grader to return **machine‑readable JSON** so that we can later **automate
aggregate scoring and analysis** (e.g., Section 5 majority vote, dashboards, or batch
experiments).

For every evaluation criterion you defined in Part 1 sectoin 3, output an object that looks
like this (keys must match your own criteria list):

* **Fluency**  
* **Grammar**  
* **Tone**  
* **Length**  
* **Grounding**  (e.g. factual alignment / no hallucinations)

We will grade every description on the **criteria** we care about for this
assignment:
```json
{
  "Fluency":   {"explanation": "...", "verdict": "good"},
  "Grammar":   {"explanation": "...", "verdict": "ok"},
  "Tone":      {"explanation": "...", "verdict": "good"},
  "Length":    {"explanation": "...", "verdict": "good"},
  "Grounding": {"explanation": "...", "verdict": "good"},
}
```


*\*The following will be ignored for convinence and as they do not require JLM*
* **Latency**
* **Cost**




### Why `{explanation, verdict}`?

1. **Justification before decision** – forcing the model to produce an *explanation*
   **first** encourages reasoned thinking; if it wrote the verdict first, the
   explanation might simply try to defend that label (confirmation bias).  
2. **Auditability** – having a short explanation lets us *spot‑check* the grader
   for hallucinations or mis‑interpretations down the line.  
3. **Closed‑class verdicts** – restricting the verdict to a small set
   (`good | ok | bad`) enables fast, deterministic score calculations in later
   automated pipelines.

### Your implementation

* **Backend choice:** `call_openai()`, `call_gemini()`, `call_hf()`, or a
  LangChain chat model – choose whatever you prefer.  
* **Prompt freedom:** use the template we provide *or rewrite it entirely*.  
* Loop over the global `criteria` list from Part 1 §3.  
* Implement **`structured_jlm(description)`** so the cell runs without errors and
  returns the parsed JSON.

In [24]:
# ------------------------- STUDENT TODO -------------------------
import json

# Fixed criteria list for the assignment
criteria = ["Fluency", "Grammar", "Tone", "Length", "Grounding"]

# Choose backend: "openai", "gemini", "hf"
BACKEND = "gemini"

# Prompt template (edit freely)
PROMPT_TEMPLATE = """You are a product‑description critic.
For each criterion in the list below, provide a JSON object with:
  • explanation (1‑2 sentences)
  • verdict      (good | ok | bad)

Criteria: {criteria}

Description:
{description}

JSON:"""

#IMPORTANT NOTE :use the same model as the one you used for generation
def call_model(prompt: str, model_name: str) -> str:
    if BACKEND == "openai":
        return call_openai(prompt, model_name)
    elif BACKEND == "gemini":
        return call_gemini(prompt, model_name)
    elif BACKEND == "hf":
        return call_hf(prompt, model_name)

def structured_jlm(description: str, model_name:str) -> dict:
    prompt = PROMPT_TEMPLATE.format(criteria=", ".join(criteria),
                                    description=description)
    raw = call_model(prompt, model_name) # NOTICE this is the raw response, modify it as you will to make the next step work

    try: #NOTE: Making sure your model returns a valid JSON object is sometimes harder than you think...
        return json.loads(raw)
    except json.JSONDecodeError:
        raise ValueError(f"Error: Invalid JSON response from model: {raw}")

# ---------------------------------------------------------------

structured_jlm(description, model_name="models/gemini-1.5-pro")

TypeError: the JSON object must be str, bytes or bytearray, not dict

## 3 Criterion‑by‑Criterion Calls

Instead of asking the JLM to grade *all* criteria in one big prompt, we can
make **one call per criterion** and merge the results.

### Why split?

1. **Sharper focus ⇢ better judgments**  
   The model concentrates on a *single* dimension at a time, which often boosts
   accuracy – atomic tasks are easier.  
2. **Richer, criterion‑specific instructions**  
   You can craft a longer, more detailed prompt for *each* criterion without
   squeezing everything into one context window.

### Trade‑offs

1. **Higher cost** – `#criteria × price‑per‑call` can add up.  
2. **Higher latency** – multiple sequential calls take longer than one.



### Your task

1. Implement **`per_criterion_jlm(description)`** (see code cell below) using any backend (`call_openai`, `call_gemini`, `call_hf`, or `jlm_llm`).
2. **Pick one generated product description** and run **all three graders**:
   * `naive_jlm(description)` (Section 1)
   * `structured_jlm(description)` (Section 2)
   * `per_criterion_jlm(description)` (this section)
3. Compare the outputs. Note any differences in explanations and verdicts.

Write two‑three sentences of reflection in the markdown cell that follows: *Which approach seems most trustworthy for this task and why?*

In [ ]:
# ------------------------- STUDENT TODO -------------------------
import json

criteria = ["Fluency", "Grammar", "Tone", "Length", "Grounding"]

# Re‑use BACKEND and call_model() from Section 2, or redefine here.
try:
    BACKEND
except NameError:
    BACKEND = "hf"  # fallback; set to openai / gemini / hf as needed

# If call_model is not defined (e.g., you skipped Section 2), define a minimal version
if "call_model" not in globals():
    def call_model(prompt: str, model_name: str) -> str:
        if BACKEND == "openai":
            return call_openai(prompt, model_name)
        elif BACKEND == "gemini":
            return call_gemini(prompt, model_name)
        elif BACKEND == "hf":
            return call_hf(prompt, model_name)

# Template per criterion (modify as you like)
CRIT_PROMPT_TEMPLATE = """You are a product‑description critic.
Criterion: {criterion}
Return JSON with keys 'explanation' and 'verdict' (good|ok|bad).

Description:
{description}

JSON:"""

#IMPORTANT NOTE :use the same model as the one you used for generation
def per_criterion_jlm(description: str, model_name: str, criteria: list[str] | None = None) -> dict:
    """Grade *description* against each criterion in *criteria* with separate LLM calls.

    Parameters
    ----------
    description : str
        Product description to evaluate.
    criteria : list[str], optional
        List of evaluation criteria. If *None*, falls back to the global ``criteria`` list
        defined in this notebook.
    Returns
    -------
    dict
        Mapping from criterion name to the parsed JSON returned by the judge‑LLM.
    """
    if criteria is None:
        # Use the global variable if available; otherwise default to common criteria.
        criteria = globals().get("criteria", ["Fluency", "Grammar", "Tone", "Length", "Grounding"])

    results = {}
    for crit in criteria:
        prompt = CRIT_PROMPT_TEMPLATE.format(criterion=crit, description=description)
        raw = call_model(prompt, model_name)
        try:
            results[crit] = json.loads(raw)
        except json.JSONDecodeError:
            raise ValueError(f"Error parsing JSON for criterion {crit}: {raw}")

    return results
# ---------------------------------------------------------------


## 4 Different Model JLM

### Why switch models?

We **do not reuse the generator model as the judge** because:

1. **Self‑evaluation bias** – a model tends to justify its own outputs, consciously or not, and can over‑score itself.  
2. **Correlated failure modes** – the generator’s blind‑spots (hallucinations, sloppy reasoning) may also appear in its judgments.  
3. **Diversity of signal** – using an *independent* model (preferably from another provider) gives a more reliable second opinion.

### Your task

* Pick a **different provider** than you used in Sections 1‑3 (e.g. if you relied on OpenAI, try Gemini or Claude; if you used HF, try OpenAI, etc.).  
* Implement **`per_criterion_jlm_alt()`** – a thin wrapper that calls `per_criterion_jlm()` *after temporarily switching* the backend to this alternate provider, so it still returns the usual `{explanation, verdict}` JSON for **each criterion**.  
  * You may use helpers such as `call_openai`, `call_gemini`, `call_hf`, or any LangChain chat model—your choice.  
  * Feel free to craft a new prompt tailored to the provider.  

### Compare

Run both graders – **Section 3’s `per_criterion_jlm()`** and the new **`per_criterion_jlm_alt()`** – on the *same* product description:

* Are the verdicts identical?  
* Does the new JLM appear to **favour or penalise** the generated answer differently?  
* Add a brief reflection in the markdown cell that follows.


In [ ]:
#TODO STUDENT: call the JLM (using section 3 code) with a different model from the one used
#      to generate the descriptions and evaluate the results so far

## 5 Creating a Dataset‑Evaluation Infrastructure

You now have several JLM graders (`structured_jlm`, `per_criterion_jlm` and `naive_jlm`) that can judge a *single* product description.  
The next step is to **scale that evaluation to an entire dataset** of model outputs so you can compute corpus‑level metrics and quickly spot problematic cases.

### Why build an infrastructure?

* **Reproducibility & tracking** – running all descriptions through the *same* grading pipeline lets you store the verdict/explanation alongside the raw generations for future audits.  
* **Bulk analytics** – with judgments in a DataFrame you can compute pass‑rates, per‑criterion confusion matrices, or slice‑by‑slice performance (e.g., electronics vs. clothing).  
* **Error triage** – the explanation text helps you cluster or search for recurring failure modes (e.g., "missing dimensions", "over‑hyping").  
* **Continuous evaluation** – once scripted, you can rerun the notebook after each model iteration and track deltas automatically.

### What to implement

1. **Iterate over the dataset** – a `pandas.DataFrame` where each row has at least a `description` column (and optionally metadata like `product_id`, `category`).  
2. **Apply your chosen grader** – call **one** of your JLM functions (`per_criterion_jlm` is a good default) per row.  
3. **Add columns** – for every evaluation criterion create `<criterion>_explanation` and `<criterion>_verdict` columns.  
4. **Save or return the enriched DataFrame** so you can inspect it, compute stats, or export to CSV/Parquet.

A starter helper function is provided in the code cell below—customise it as needed.

In [ ]:
# ------------------------- STUDENT TODO -------------------------
# Iterate over a DataFrame of product descriptions, run alt_jlm on each row,
# and append explanation & verdict columns per criterion.
#
# Expected input format:
#   df["description"]  -> the text to judge
#
# Example output columns for criterion 'clarity':
#   clarity_explanation , clarity_verdict
#
# Adjust variable names / saving as needed.

import pandas as pd

def judge_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    judged_rows = []
    for _, row in df.iterrows():
        desc = row["description"]
        result = per_criterion_jlm(desc, criteria)  # {crit: {"explanation": ..., "verdict": ...}}
        flat = row.to_dict()
        for crit, vals in result.items():
            flat[f"{crit}_explanation"] = vals["explanation"]
            flat[f"{crit}_verdict"] = vals["verdict"]
        judged_rows.append(flat)
    return pd.DataFrame(judged_rows)

# Example usage:
# df_input = pd.read_csv("my_descriptions.csv")
# df_scored = judge_dataframe(df_input)
# df_scored.to_csv("descriptions_with_judgments.csv", index=False)
# df_scored.head()
# ---------------------------------------------------------------

judge_dataframe("YOUR_DF")



You now have a DataFrame with per‑criterion **explanations** and **verdicts**.
Next, add a final scoring pass that condenses those signals into one overall
label (`good | ok | bad`) per description.

### Judgement guidelines object

Provide a dict with **exactly these criteria** and a Boolean indicating whether
the criterion is a **no‑go** (i.e. a *bad* verdict forces the overall label to
*bad*):

| Criterion   | No‑go? |
|-------------|--------|
| Fluency     | False  |
| Grammar     | True?  |
| Tone        | False  |
| Length      | False  |
| Grounding   | True?  |

Adjust the True/False flags to suit your project.

```python
judgement_guidelines = {
    "Fluency":   False,
    "Grammar":   True,
    "Tone":      False,
    "Length":    False,
    "Grounding": True,
}
```

### Task

1. Implement `apply_final_scoring(df, judgement_guidelines)` (cell below).  
2. Ensure it adds a `final_label` column using no‑go rules plus an aggregated
   score from the remaining criteria.

In [ ]:
# ------------------------- STUDENT TODO -------------------------
# Post‑processing: compute a row‑level final score using guideline rules.
# Inputs:
#   df                – the DataFrame returned by judge_dataframe()
#   judgement_guidelines – dict mapping criterion -> bool
#                          True  => "no‑go" criterion (a 'bad' verdict => overall bad)
#                          False => normal criterion
#
# Strategy (example – feel free to tweak):
#   • Map verdicts to numeric scores {good: 2, ok: 1, bad: 0}.
#   • If any no‑go criterion is 'bad' => overall_label = 'bad' (score 0).
#   • Otherwise sum numeric scores, normalise by (2 * #criteria) to get 0‑1 range,
#       and convert back to overall_label ('good' ≥ 0.75, 'ok' ≥ 0.4 else 'bad').

VERDICT_TO_SCORE = {"good": 2, "ok": 1, "bad": 0}

def apply_final_scoring(df, judgement_guidelines: dict) -> pd.DataFrame:
    scored_rows = []
    crits = [c for c in judgement_guidelines.keys()]
    for _, row in df.iterrows():
        # Check no‑go first
        overall_label = None
        for crit in crits:
            v = row.get(f"{crit}_verdict", "").lower()
            if judgement_guidelines[crit] and v == "bad":
                overall_label = "bad"
                break
        # If not nailed by no‑go, compute aggregate
        if overall_label is None:
            total = 0
            max_total = 2 * len(crits)
            for crit in crits:
                total += VERDICT_TO_SCORE.get(row.get(f"{crit}_verdict", "").lower(), 0)
            norm = total / max_total
            if norm >= 0.75:
                overall_label = "good"
            elif norm >= 0.4:
                overall_label = "ok"
            else:
                overall_label = "bad"
        scored_row = row.to_dict()
        scored_row["final_label"] = overall_label
        scored_rows.append(scored_row)
    return pd.DataFrame(scored_rows)

# Example usage:
# guidelines = {"clarity": False, "factuality": True, "persuasiveness": False}
# df_scored = apply_final_scoring(df_scored, guidelines)
# df_scored.head()
# ---------------------------------------------------------------

apply_final_scoring("YOUR_JUDGED_DF", "YOUR_GUIDELINES")

## 6 Majority‑Vote Ensemble - NOT EASY

Ensembling is a simple way to boost robustness.  
Here we will take **multiple independent calls** to the **alternate judge model** you built in Section 4 (`per_criterion_jlm`) and
combine their verdicts with a majority vote.

*Call *different* `models` several times so that each run can produce (hopefully) different explanations/verdicts.*

### Your task

1. Implement **`majority_vote_jlm(description, n=3)`** that:
   * Invokes `per_criterion_jlm(description)` with **n** different models (minimum of 3),
   * Return a dictionary with all results that will later be written to a pd DF.
2. Test it on the same product description you used in Sections 3–4.
3. Briefly comment: *Does majority voting smooth out noisy judgments?*

In [ ]:
#NOTE : Call at least 3 different models - not same model, return all responses and then take majority vote

# ------------------------- STUDENT TODO -------------------------

def majority_vote_jlm(description: str, n: int = 3) -> dict:
    #TODO : Implement majority vote
    raise NotImplementedError("Implement majority_vote_jlm")
# ---------------------------------------------------------------

majority_vote_jlm("SOME_DESCRIPTION", n)

## 7 *(Bonus)* Dataset‑Level Majority‑Vote Evaluation

You have a row‑level ensemble grader (`majority_vote_jlm`) that produces a more
stable verdict for **one** product description.  
In this bonus section you’ll **scale** that ensemble to an entire DataFrame so
you can compute dataset‑level metrics.

### Goals

1. **Automate majority voting across the dataset**: run `majority_vote_jlm`
   *n* times per description and store the combined verdicts/explanations.  
2. **Reuse your scoring pipeline**: pipe the enriched DataFrame through
   `apply_final_scoring` to get a `final_label`.  
3. **Compare**: how do the label distributions differ from the single‑judge
   evaluation in Section 5?

> *Extra credit*: add timing & cost tracking so you can quantify the trade‑off.

In [ ]:
# ------------------------- BONUS TODO -------------------------
import pandas as pd

def judge_dataframe_mv(df: pd.DataFrame, n: int = 3) -> pd.DataFrame:
    """Run majority‑vote JLM `n` times per description and return enriched DF."""
    rows = []
    for _, row in df.iterrows():
        desc = row["description"]
        result = majority_vote_jlm(desc, n=n)
        flat = row.to_dict()
        for crit, vals in result.items():
            flat[f"{crit}_explanation"] = vals["explanation"]
            flat[f"{crit}_verdict"] = vals["verdict"]
        rows.append(flat)
    return pd.DataFrame(rows)

# Example workflow:
# df_mv = judge_dataframe_mv(df_input, n=5)
# df_mv_scored = apply_final_scoring(df_mv, judgement_guidelines)
# print(df_mv_scored['final_label'].value_counts())
# ---------------------------------------------------------------
